In [4]:
# -*- coding: utf-8 -*-
"""nn

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1mED2_1ArZUU_cwO9x5rhRG1VRQldrL4p
"""

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Embedding, LSTM, Dense, Dropout, Bidirectional, Conv2D, MaxPooling2D, Flatten,
    BatchNormalization, GlobalMaxPooling1D, Conv1D, Input, Normalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import TimeDistributed, Dense
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.losses import Huber
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import GRU
import tensorflow.keras as keras

def create_toy_rnn(input_shape, n_outputs):
    print("Shape of input data:", input_shape)

    model = keras.Sequential([
        keras.layers.Input(shape=input_shape),  #Explicit Input layer
        keras.layers.SimpleRNN(128, activation="tanh", return_sequences=True,
                               kernel_initializer="glorot_uniform",
                               kernel_regularizer=keras.regularizers.l2(0.001)),  # L2 regularization
        keras.layers.Dropout(0.1),  #Lower dropout to retain more info
        keras.layers.SimpleRNN(64, activation="tanh", return_sequences=True),
        keras.layers.Dense(32, activation="relu"),  #Extra dense layer for better feature extraction
        keras.layers.Dense(n_outputs, activation="linear")
    ])
    optimizer = keras.optimizers.RMSprop(learning_rate=0.002)  #RMSprop is often better for RNNs
    model.compile(loss="mse", optimizer=optimizer)
    return model, {}

def create_mnist_cnn(input_shape: tuple, n_outputs: int):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=(3, 3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)  #Reduced dropout for better learning
    outputs = Dense(n_outputs, activation="softmax")(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model, {"epochs": 12, "batch_size": 32}

def create_youtube_comment_rnn(vocabulary, n_outputs):
    vocab_size = len(vocabulary) + 1
    embedding_dim = 128
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True),
        SpatialDropout1D(0.3),
        Bidirectional(LSTM(128, return_sequences=True)),
        GRU(64, return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(n_outputs, activation='sigmoid' if n_outputs == 1 else 'softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss="binary_crossentropy" if n_outputs == 1 else "categorical_crossentropy",
                  metrics=["accuracy"])

    return model, {"epochs": 12, "batch_size": 32}

def create_youtube_comment_cnn(vocabulary, n_outputs):
    inputs = Input(shape=(None,))
    x = Embedding(input_dim=len(vocabulary) + 1, output_dim=64)(inputs)
    x = Conv1D(64, kernel_size=3, activation="relu")(x)
    x = GlobalMaxPooling1D()(x)
    outputs = Dense(n_outputs, activation="sigmoid" if n_outputs == 1 else "softmax")(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy" if n_outputs == 1 else "categorical_crossentropy", metrics=["accuracy"])
    return model, {"epochs": 10, "batch_size": 32}